In [ ]:
# connect to google drive to access the data files
# You will need to give premission to access your drive
# if this is the first time doing so

from google.colab import drive
drive.mount('/content/drive')

# Create paths to file locations
# Update these paths to where your store the files in your drive
Lead_path = '/content/drive/MyDrive/LSC 586/Arabidopsis Lead Treatment.gz'
Control_Path = '/content/drive/MyDrive/LSC 586/Arabidopsis Untreated Control.gz'
Vanadium_Path = '/content/drive/MyDrive/LSC 586/Arabidopsis Vanadium Treatment.gz'

fasta_path = '/content/drive/MyDrive/LSC 586/GCF_000001735.4_TAIR10.1_genomic.fna'
gtf_gz_path = '/content/drive/MyDrive/LSC 586/GCF_000001735.4_TAIR10.1_genomic.gtf.gz'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Installtion with STAR

STAR is an ultra fast universal RNA-seq alignment tool

Paper: https://pmc.ncbi.nlm.nih.gov/articles/PMC3530905/

### Tutorial:

Here: https://hbctraining.github.io/Intro-to-rnaseq-hpc-O2/lessons/03_alignment.html

In [ ]:
# install STAR

# Step 1: Download the STAR source tarball
star_tarball_url = "https://github.com/alexdobin/STAR/archive/2.7.9a.tar.gz"
tarball_name = "STAR-2.7.9a.tar.gz"

print("Downloading STAR source code...")
subprocess.run(["wget", star_tarball_url, "-O", tarball_name], check=True)

# Step 2: Extract the tarball
print("Extracting STAR source code...")
subprocess.run(["tar", "-xzvf", tarball_name], check=True)

# Step 3: Compile STAR
star_source_dir = "STAR-2.7.9a/source"
print("Compiling STAR...")
subprocess.run(["make", "-C", star_source_dir], check=True)

# Confirm that the STAR binary exists
star_binary = os.path.join(star_source_dir, "STAR")
if os.path.exists(star_binary):
    print(f"STAR compiled successfully! Binary is located at: {star_binary}")
else:
    print("Compilation failed: STAR binary not found.")


Extracting STAR source code...
Compiling STAR...
STAR compiled successfully! Binary is located at: STAR-2.7.9a/source/STAR


In [ ]:
!apt-get install -y samtools subread

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
subread is already the newest version (2.0.3+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [ ]:
# import required packages
import gzip
import shutil
import os
import subprocess

# 1. Unzip and decompress Data sets

In [ ]:
def decompress_gz(input_path, output_path):
    """Decompress a .gz file to the specified output file."""
    with gzip.open(input_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"Decompressed {input_path} to {output_path}")

# Create a directory for decompressed data
os.makedirs("raw_data", exist_ok=True)

# Define gzipped files and desired output FASTQ
files = {
    Lead_path: "Arabidopsis_Lead_Treatment.fastq",
    Control_Path: "Arabidopsis_Untreated_Control.fastq",
    Vanadium_Path: "Arabidopsis_Vanadium_Treatment.fastq"
}

# Decompress and save to raw_data directory
for gz_file, fastq_file in files.items():
    input_path = gz_file
    output_path = os.path.join("raw_data", fastq_file)
    decompress_gz(input_path, output_path)

Decompressed /content/drive/MyDrive/LSC 586/Arabidopsis Lead Treatment.gz to raw_data/Arabidopsis_Lead_Treatment.fastq
Decompressed /content/drive/MyDrive/LSC 586/Arabidopsis Untreated Control.gz to raw_data/Arabidopsis_Untreated_Control.fastq
Decompressed /content/drive/MyDrive/LSC 586/Arabidopsis Vanadium Treatment.gz to raw_data/Arabidopsis_Vanadium_Treatment.fastq


# 2. Create Index for alignment on Reference Genome

In [ ]:
def decompress_gtf(gz_file, output_file):
    """
    Decompress a gzipped GTF file.
    """
    with gzip.open(gz_file, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"Decompressed {gz_file} to {output_file}")

# Define file paths
fasta_file = fasta_path
gtf_gz_file = gtf_gz_path
gtf_file = "GCF_000001735.4_TAIR10.1_genomic.gtf"  # Decompressed file output
star_index_dir = "star_index"

# Check if the GTF file has been decompressed
if not os.path.exists(gtf_file):
    decompress_gtf(gtf_gz_file, gtf_file)

# Create the STAR index directory if it doesn't exist
# This step is crucial for STAR to work properly
os.makedirs(star_index_dir, exist_ok=True)

# Define parameters
threads = "4"
sjdbOverhang = "99"  # read length minus one but we can adjust this as needed

# Build the STAR index command
star_command = [
    "STAR-2.7.9a/source/STAR",
    "--runThreadN", threads,
    "--runMode", "genomeGenerate",
    "--genomeDir", star_index_dir,
    "--genomeFastaFiles", fasta_file,
    "--sjdbGTFfile", gtf_file,
    "--sjdbOverhang", sjdbOverhang
]

print("Building STAR index with the following command:")
print(" ".join(star_command))

# Run STAR command to generate the index
subprocess.run(star_command, check=True)

print("STAR index built successfully!")


Building STAR index with the following command:
STAR-2.7.9a/source/STAR --runThreadN 4 --runMode genomeGenerate --genomeDir star_index --genomeFastaFiles /content/drive/MyDrive/LSC 586/GCF_000001735.4_TAIR10.1_genomic.fna --sjdbGTFfile GCF_000001735.4_TAIR10.1_genomic.gtf --sjdbOverhang 99
STAR index built successfully!


# 3. Align data samples to indexed reference genome

In [ ]:
# This cell will take approximately 2 hours to run given the 3 files below

# List of FASTQ files (from "raw_data" directory)
samples = [
    "Arabidopsis_Lead_Treatment.fastq",
    "Arabidopsis_Untreated_Control.fastq",
    "Arabidopsis_Vanadium_Treatment.fastq"
]

raw_data_dir = "raw_data"
alignment_dir = "alignments_star"
os.makedirs(alignment_dir, exist_ok=True)
threads = "4"
star_index_dir = "star_index"

# The absolute path to your STAR binary (from the previous step)
star_binary = "./STAR-2.7.9a/source/STAR"

for sample in samples:
    fastq_path = os.path.join(raw_data_dir, sample)
    # Set an output prefix for STAR
    output_prefix = os.path.join(alignment_dir, sample.replace(".fastq", "_"))

    # Build STAR command for aligning the reads
    star_command = [
        star_binary,
        "--runThreadN", threads,
        "--genomeDir", star_index_dir,
        "--readFilesIn", fastq_path,
        "--outFileNamePrefix", output_prefix
    ]

    print(f"Aligning sample {sample}...")
    subprocess.run(star_command, check=True)
    print(f"Alignment for {sample} completed. Output files begin with: {output_prefix}")


Aligning sample Arabidopsis_Lead_Treatment.fastq...
Alignment for Arabidopsis_Lead_Treatment.fastq completed. Output files begin with: alignments_star/Arabidopsis_Lead_Treatment_
Aligning sample Arabidopsis_Untreated_Control.fastq...
Alignment for Arabidopsis_Untreated_Control.fastq completed. Output files begin with: alignments_star/Arabidopsis_Untreated_Control_
Aligning sample Arabidopsis_Vanadium_Treatment.fastq...
Alignment for Arabidopsis_Vanadium_Treatment.fastq completed. Output files begin with: alignments_star/Arabidopsis_Vanadium_Treatment_


# 4. Convert SAM to BAM

After the alignment, SAM files will be created. We need to convert these to BAM files and sort them in order to get a gene expression count

In [ ]:
# Directories for alignment outputs and BAM files
alignment_dir = "alignments_star"
bam_dir = "bam_files"
os.makedirs(bam_dir, exist_ok=True)

# List SAM file names as generated by STAR
# These should be located in the alignments_star folder that was created.
# If not update below to their paths
samples = [
    "/content/alignments_star/Arabidopsis_Lead_Treatment_Aligned.out.sam",
    "/content/alignments_star/Arabidopsis_Untreated_Control_Aligned.out.sam",
    "/content/alignments_star/Arabidopsis_Vanadium_Treatment_Aligned.out.sam"
]

# Convert SAM to BAM and sort
for sample in samples:
    sam_file = os.path.join(alignment_dir, sample)
    bam_file = os.path.join(bam_dir, sample.replace(".sam", ".bam"))
    sorted_bam_file = os.path.join(bam_dir, sample.replace(".sam", "_sorted.bam"))

    # Convert SAM to BAM
    print(f"Converting {sam_file} to BAM...")
    subprocess.run(["samtools", "view", "-bS", sam_file, "-o", bam_file], check=True)

    # Sort the BAM file
    print(f"Sorting {bam_file}...")
    subprocess.run(["samtools", "sort", bam_file, "-o", sorted_bam_file], check=True)

    print(f"Finished processing {sample}. Sorted BAM: {sorted_bam_file}")

# Count reads per gene using featureCounts
# List all sorted BAM files
sorted_bam_files = [os.path.join(bam_dir, sample.replace(".sam", "_sorted.bam")) for sample in samples]

# annotation file (GTF) and output file for gene counts
annotation_file = "GCF_000001735.4_TAIR10.1_genomic.gtf"
output_counts = "gene_counts.txt"

# Build the featureCounts command:
# -T: number of threads
# -a: annotation file (GTF)
# -o: output file for counts
# Followed by a list of BAM files.
featurecounts_command = [
    "featureCounts",
    "-T", "4",
    "-a", annotation_file,
    "-o", output_counts
] + sorted_bam_files

print("Running featureCounts to generate gene count matrix...")
subprocess.run(featurecounts_command, check=True)
print(f"Gene counts generated and saved in {output_counts}")


Converting /content/alignments_star/Arabidopsis_Lead_Treatment_Aligned.out.sam to BAM...
Sorting /content/alignments_star/Arabidopsis_Lead_Treatment_Aligned.out.bam...
Finished processing /content/alignments_star/Arabidopsis_Lead_Treatment_Aligned.out.sam. Sorted BAM: /content/alignments_star/Arabidopsis_Lead_Treatment_Aligned.out_sorted.bam
Converting /content/alignments_star/Arabidopsis_Untreated_Control_Aligned.out.sam to BAM...
Sorting /content/alignments_star/Arabidopsis_Untreated_Control_Aligned.out.bam...
Finished processing /content/alignments_star/Arabidopsis_Untreated_Control_Aligned.out.sam. Sorted BAM: /content/alignments_star/Arabidopsis_Untreated_Control_Aligned.out_sorted.bam
Converting /content/alignments_star/Arabidopsis_Vanadium_Treatment_Aligned.out.sam to BAM...
Sorting /content/alignments_star/Arabidopsis_Vanadium_Treatment_Aligned.out.bam...
Finished processing /content/alignments_star/Arabidopsis_Vanadium_Treatment_Aligned.out.sam. Sorted BAM: /content/alignments

# 5. Generate gene count file to be used in Differential Expression Analysis

In [ ]:
# Directory where sorted BAM files are stored
bam_dir = "/content/alignments_star"

# List sorted BAM files (assumes they end with "_sorted.bam")
sorted_bam_files = [os.path.join(bam_dir, f) for f in os.listdir(bam_dir) if f.endswith("_sorted.bam")]
print("Sorted BAM files:", sorted_bam_files)

# annotation file (GTF format)
annotation_file = "GCF_000001735.4_TAIR10.1_genomic.gtf"

# output file for gene counts
output_counts = "gene_counts.txt"

# Build the featureCounts command:
# -T: Number of threads to use
# -a: Annotation file (GTF)
# -o: Output file for the count matrix
featurecounts_command = [
    "featureCounts",
    "-T", "4",
    "-a", annotation_file,
    "-o", output_counts
] + sorted_bam_files

print("Running featureCounts with the following command:")
print(" ".join(featurecounts_command))

# Generate gene counts
try:
    subprocess.run(featurecounts_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(f"Gene counts generated successfully and saved in {output_counts}")
except subprocess.CalledProcessError as e:
    print("Error during featureCounts execution:")
    print(e.stderr.decode())

# OPTIONAL: Save results to csv file
# Read the gene counts file while skipping comment lines (those starting with '#')
df = pd.read_csv(output_counts, sep="\t", comment="#")

# Write the DataFrame to a CSV file
df.to_csv("gene_counts.csv", index=False)
print("Gene counts saved to gene_counts.csv")

Sorted BAM files: ['/content/alignments_star/Arabidopsis_Untreated_Control_Aligned.out_sorted.bam', '/content/alignments_star/Arabidopsis_Lead_Treatment_Aligned.out_sorted.bam', '/content/alignments_star/Arabidopsis_Vanadium_Treatment_Aligned.out_sorted.bam']
Running featureCounts with the following command:
featureCounts -T 4 -a GCF_000001735.4_TAIR10.1_genomic.gtf -o gene_counts.txt /content/alignments_star/Arabidopsis_Untreated_Control_Aligned.out_sorted.bam /content/alignments_star/Arabidopsis_Lead_Treatment_Aligned.out_sorted.bam /content/alignments_star/Arabidopsis_Vanadium_Treatment_Aligned.out_sorted.bam
Gene counts generated successfully and saved in gene_counts.txt
